# LSLGA Galaxy Pairs

Identify galaxies *not* in the LSLGA which are in the "ellipse of influence" of a galaxy *in* the LSLGA.  It's important to identify and handle this case because non-LSGLA galaxies in the (elliptical) footprint of an LSLGA galaxy are forced to be PSF.

Note: in DR8 the LSLGA was limited to galaxies with angular diameters > 20 arcsec (N~500k). In DR9 we are experimenting with a larger cut, 60 arcsec (N~20k), which should mitigate this issue. Nevertheless, it's important to capture this case.

In [1]:
import os, pdb
import numpy as np

In [2]:
from astrometry.libkd.spherematch import tree_build_radec, tree_search_radec

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [4]:
import LSLGA.io

Read the LSLGA and the parent HyperLEDA sample.

In [5]:
def read_leda():
    leda = LSLGA.io.read_hyperleda(verbose=True)
    kd = tree_build_radec(leda['RA'], leda['DEC'] )
    return leda, kd

In [6]:
%time leda, kdleda = read_leda()

Read 1436176 objects from /Users/ioannis/research/projects/LSLGA/sample/v3.0/hyperleda-d25min10-18nov14.fits
Read 1436176 objects from /Users/ioannis/research/projects/LSLGA/sample/v3.0/hyperleda-d25min10-18nov14-allwise.fits
  Identified 1379531/1436176 (96.06%) objects with AllWISE photometry.
CPU times: user 2.17 s, sys: 873 ms, total: 3.04 s
Wall time: 3.07 s


In [7]:
lslga = LSLGA.io.read_parent(verbose=True)
lslga = lslga[lslga['IN_DESI']]

Read galaxy indices 0 through 21426 (N=21427) from /Users/ioannis/research/projects/LSLGA/sample/v3.0/LSLGA-v3.0.fits


In [35]:
lslga[np.argsort(lslga['D25'])[::-1]]

LSLGA_ID,GALAXY,PGC,RA,DEC,TYPE,BAR,RING,MULTIPLE,COMPACTNESS,T,PA,D25,BA,DIAM_REF,Z,SB_D25,MAG,MAG_REF,WISE_RA,WISE_DEC,CNTR,W1MPRO,W1SIGMPRO,W2MPRO,W2SIGMPRO,W3MPRO,W3SIGMPRO,W4MPRO,W4SIGMPRO,RCHI2,CC_FLAGS,EXT_FLG,PH_QUAL,XSCPROX,W1RSEMI,W1BA,W1PA,W1GMAG,W1GERR,W2GMAG,W2GERR,W3GMAG,W3GERR,W4GMAG,W4GERR,IN_ALLWISE,IN_DESI,NEAR_BRIGHTSTAR
int64,bytes29,int64,float64,float64,bytes4,bytes3,bytes3,bytes3,bytes3,float32,float32,float32,float32,bytes3,float32,float32,float32,bytes1,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float32,bytes4,int32,bytes4,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bool,bool,bool
972325,NGC0598,5818,23.462120999999996,30.659925199999996,Sc,nan,nan,nan,nan,5.9,22.67,62.086903,0.5915616,iso,-0.00060808734,23.94448,6.351,B,23.462066,30.660195,226130201351047860,10.639,0.025,10.652,0.024,9.767,0.104,6.544,0.081,7.489,0000,5,AAAA,0.92,180.0,0.88,13.0,6.302,0.016,6.312,0.02,3.157,0.045,1.232,0.052,True,True,False
1237406,NGC5457,50063,210.80236799999997,54.349022999999995,SABc,B,nan,M,nan,5.9,nan,23.988337,0.9616123,iso,0.0007902133,23.88948,8.361,B,210.8021726,54.3487903,2119154501351013505,10.348,0.023,10.213,0.021,6.624,0.018,4.453,0.029,33.86,0000,5,AAAA,0.56,180.0,0.98,28.0,6.751,0.019,6.704,0.019,3.397,0.024,1.778,0.104,True,True,False
336131,NGC4236,39346,184.1754435,69.4626085,SBd,B,nan,nan,nan,8.0,161.33,23.496332,0.29174268,iso,-1.2341871e-05,25.540482,10.057,B,184.1748933,69.4629059,1841169601351018960,12.94,0.024,13.034,0.026,11.564,0.152,8.623,nan,2.984,0000,5,AABU,1.42,73.49,0.54,-42.0,9.999,0.011,10.067,0.014,8.144,0.039,6.082,0.14,True,True,False
1314798,NGC3031,28630,148.88815649999998,69.0652757,Sab,nan,nan,nan,nan,2.4,157.0,21.627186,0.519996,iso,-0.0001264208,23.094482,7.791,B,148.8884508,69.0652157,1507169601351009653,5.742,0.038,6.472,0.019,5.092,0.014,2.957,0.021,48.43,0000,5,AAAA,0.3,180.0,0.58,-38.0,4.377,0.011,4.429,0.012,3.318,0.012,1.99,0.021,True,True,False
510364,NGC2403,21396,114.2142375,65.60242029999999,SABc,B,nan,nan,nan,6.0,126.33,19.952621,0.5046613,iso,0.00046999176,23.954481,8.826,B,0.0,0.0,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,,0,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,True,False
965484,IC1613,3844,16.199166,2.1177314,I,B,nan,nan,nan,9.9,nan,18.323145,0.9354057,iso,-0.00077253446,24.952482,10.009,B,0.0,0.0,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,,0,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,True,False
669240,NGC4258,39600,184.7399355,47.3038816,Sbc,B,nan,M,nan,4.0,150.0,16.982437,0.4265795,iso,0.0015150481,23.90248,9.124,B,184.7395849,47.3040128,1854146901351040260,8.53,0.023,8.19,0.02,5.482,0.015,3.328,0.02,27.52,0000,5,AAAA,1.62,180.0,0.8,-25.0,5.599,0.019,5.578,0.02,3.073,0.008,1.353,0.015,True,True,False
52700,NGC4565,42038,189.086598,25.9875214,Sb,nan,nan,nan,nan,3.2,135.2,16.749428,0.17258379,iso,0.0040894956,24.803482,10.055,B,189.0865292,25.9876632,1883125701351051133,7.96,0.023,8.201,0.019,6.572,0.014,4.938,0.032,56.18,0000,5,AAAA,1.1,180.0,0.61,-44.0,5.935,0.005,5.883,0.006,3.472,0.006,1.932,0.019,True,True,False
526710,NGC4244,39422,184.37279549999997,37.807792799999994,Sc,nan,nan,nan,nan,6.0,44.77,16.218103,0.4466836,iso,0.0008178991,25.117481,10.439,B,184.3728014,37.8074324,1837137801351030161,11.795,0.023,11.835,0.024,9.271,0.038,7.053,0.114,31.26,0000,5,AAAB,2.5,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,True,True,False


In [31]:
indx = np.zeros(len(lslga), dtype=bool)
for ii, gg in enumerate(lslga[:1000]):
    I = tree_search_radec(kdleda, gg['RA'], gg['DEC'], gg['D25'] / 1.8 / 60) # 1.8 instead of 2...
    if len(I) > 2: # ignore the LSLGA galaxy itself
        #indx[I] = True
        print(lslga['GALAXY'][ii])
        for _I in I:
            print('  {} {:.3f}'.format(leda['GALAXY'][_I], 60*leda['D25'][_I]))
        print()
#print('Found {}/{} galaxies inside an LSLGA.'.format(np.sum(indx), len(lslga)))

NGC5661
  NGC5661 68.573
  PGC051924 16.412
  SDSSJ143157.54+061448.2 57.697

UGC06207
  UGC06207 91.443
  SDSSJ110955.57+241534.4 59.334
  UGC06204 58.499

IC1101
  IC1101 71.804
  SDSSJ151054.50+054434.3 10.451
  SDSSJ151056.40+054505.5 39.825

NGC5778
  SDSSJ145433.31+183816.8 21.486
  SDSSJ145433.70+183819.9 12.652
  NGC5778 73.986

PGC042347
  PGC042346 92.929
  PGC042347 70.494
  SDSSJ124345.48+545329.4 20.948

NGC4565
  SDSSJ123643.42+260647.1 13.809
  NGC4565 1004.966
  PGC2793674 36.237

UGC09117
  PGC050888 17.424
  UGC09117 100.728
  PGC050887 18.886

NGC7535
  SDSSJ231411.67+133447.0 78.732
  NGC7535 87.731
  SDSSJ231413.99+133458.5 60.138

NGC7649
  PGC1463038 18.161
  NGC7649 97.533
  SDSSJ232417.99+143859.9 12.857
  SDSSJ232418.38+143913.7 25.595

NGC4559
  SDSSJ123600.18+275225.1 12.392
  NGC4559 634.090
  2MASXJ12360871+2757561 17.830
  SDSSJ123534.85+275919.3 14.796

NGC5789
  NGC5789 69.207
  SDSSJ145636.07+301411.3 75.710
  SDSSJ145634.31+301421.0 38.918

UGC09977
 

In [ ]:
stop

In [32]:
isdss = ['SDSS' in gg for gg in lslga['GALAXY']]
lslga[isdss]

LSLGA_ID,GALAXY,PGC,RA,DEC,TYPE,BAR,RING,MULTIPLE,COMPACTNESS,T,PA,D25,BA,DIAM_REF,Z,SB_D25,MAG,MAG_REF,WISE_RA,WISE_DEC,CNTR,W1MPRO,W1SIGMPRO,W2MPRO,W2SIGMPRO,W3MPRO,W3SIGMPRO,W4MPRO,W4SIGMPRO,RCHI2,CC_FLAGS,EXT_FLG,PH_QUAL,XSCPROX,W1RSEMI,W1BA,W1PA,W1GMAG,W1GERR,W2GMAG,W2GERR,W3GMAG,W3GERR,W4GMAG,W4GERR,IN_ALLWISE,IN_DESI,NEAR_BRIGHTSTAR
int64,bytes29,int64,float64,float64,bytes4,bytes3,bytes3,bytes3,bytes3,float32,float32,float32,float32,bytes3,float32,float32,float32,bytes1,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float32,bytes4,int32,bytes4,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bool,bool,bool
1687,SDSSJ140511.30+060559.4,5166527,211.297104,6.0998595,nan,nan,nan,nan,nan,nan,171.52,1.9783283,0.16046767,r,0.28075022,25.811972,15.702,B,0.0,0.0,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,,0,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,True,True
4337,SDSSJ083131.00+192042.6,4168240,127.879266,19.3451956,S?,nan,nan,nan,nan,2.1,146.23,1.0280162,0.13708818,iso,0.039242815,26.41748,17.729,B,127.8791702,19.3451065,1280119601351018561,14.456,0.035,14.252,0.056,11.427,0.226,8.732,nan,1.74,0000,1,AABU,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,True,True,False
34137,SDSSJ160938.64+463418.0,4016359,242.410953,46.571725799999996,Sm,nan,nan,nan,nan,9.0,51.94,1.164126,0.15452544,iso,0.009719056,26.325481,17.367,B,242.4115568,46.5719918,2421146901351018361,15.97,0.039,15.943,0.097,13.32,nan,9.822,nan,1.078,0000,0,AAUU,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,True,True,False
84601,SDSSJ121607.89+403649.0,4548454,184.03303799999998,40.6136976,nan,nan,nan,nan,nan,nan,19.63,2.344229,0.8790225,iso,0.39469638,22.505482,12.027,B,0.0,0.0,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,,0,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,True,False
91161,SDSSJ135302.77+123311.7,4012202,208.26156,12.5533231,S?,nan,nan,nan,nan,10.0,127.2,1.0280162,0.29853827,iso,0.02179341,25.63348,16.945,B,0.0,0.0,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,,0,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,True,False
94112,SDSSJ020334.60-092540.0,4007701,30.894252,-9.427824900000001,nan,nan,nan,nan,nan,nan,74.9,1.5922089,0.57016426,iso,0.34571248,23.814482,14.176,B,30.8941679,-9.4277771,306009101351021599,15.371,0.038,15.293,0.084,12.015,0.25,8.825,nan,0.9982,hh00,0,AABU,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,True,True,False
94322,SDSSJ015032.27+133942.2,4007659,27.634411500000002,13.6617434,S?,nan,nan,nan,nan,5.0,105.46,1.0914403,0.15205474,iso,0.017361343,26.456482,17.638,B,27.6347237,13.6617552,279113601351030238,14.673,0.031,14.612,0.053,11.935,0.237,8.413,nan,1.462,0000,0,AABU,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,True,True,False
98779,SDSSJ110439.61+274325.9,4581667,166.1650515,27.7239206,Sc,nan,nan,nan,nan,5.7,56.03,1.3182567,0.66069347,iso,0.008535905,24.792482,15.564,B,166.165727,27.7238121,1656127201351046153,14.065,0.028,13.889,0.039,11.68,0.253,8.79,0.517,3.993,d000,1,AABC,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,True,True,False
103980,SDSSJ135737.29+392853.5,4012235,209.40535049999997,39.481546,I,nan,nan,nan,nan,10.0,171.71,1.0280162,0.24547088,iso,0.004594845,25.850481,17.162,B,209.4054362,39.4817165,2090139301351030542,16.016,0.052,15.731,0.122,12.4,nan,8.804,nan,1.014,0000,0,ABUU,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,True,True,False


In [ ]:
plt.scatter(cat['MAG'], cat['D25'], s=5)
plt.yscale('log')